<a href="https://colab.research.google.com/github/rishi12jn/Cyber-Security-Lab/blob/main/cyber_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install scapy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.0 MB/s eta 0:00:00


In [ ]:
from scapy.all import rdpcap, TCP, UDP, ICMP

# Path to the pcap file
pcap_file = "/content/Sample.pcap"  # Change this to your actual file path

# Read the packets from the pcap file
packets = rdpcap(pcap_file)

# Initialize counters
tcp_count = 0
udp_count = 0
icmp_count = 0

# Iterate through packets and count protocol occurrences
for packet in packets:
    if packet.haslayer(TCP):
        tcp_count += 1
    elif packet.haslayer(UDP):
        udp_count += 1
    elif packet.haslayer(ICMP):
        icmp_count += 1

# Display the results
print(f"TCP Packets: {tcp_count}")
print(f"UDP Packets: {udp_count}")
print(f"ICMP Packets: {icmp_count}")

TCP Packets: 2093
UDP Packets: 393
ICMP Packets: 0


In [ ]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.9 MB/s eta 0:00:00


In [5]:
from scapy.all import rdpcap, IP, TCP

# Path to the pcap file
pcap_file_path = "/content/Sample.pcap"

# Read packets from the pcap file
packets = rdpcap(pcap_file_path)

# Dictionaries to track request counts and failed login attempts
ip_request_count = {}
failed_login_attempts = {}

# Authentication-related ports
login_ports = {21: "FTP", 22: "SSH", 23: "Telnet", 25: "SMTP", 80: "HTTP", 443: "HTTPS", 3389: "RDP", 3306: "MySQL"}

# Analyze packets
for packet in packets:
    if packet.haslayer(IP) and packet.haslayer(TCP):
        ip_src = packet[IP].src
        dst_port = packet[TCP].dport
        tcp_flags = packet[TCP].flags

        # Track general request count per IP
        ip_request_count[ip_src] = ip_request_count.get(ip_src, 0) + 1

        # Detect failed logins based on TCP behavior
        if dst_port in login_ports:
            # Failed login pattern: SYN is sent but gets a RST (reset) response
            if tcp_flags == "S":  # Initial SYN packet
                failed_login_attempts[ip_src] = failed_login_attempts.get(ip_src, 0) + 1

# Define detection thresholds
brute_force_threshold = 100
failed_login_threshold = 50

# Identify brute-force attack sources
brute_force_ips = [ip for ip, count in ip_request_count.items() if count > brute_force_threshold]

# Identify excessive failed login attempts
failed_login_ips = [ip for ip, count in failed_login_attempts.items() if count > failed_login_threshold]

# Display results
print("\n========= DETECTION REPORT =========")
print("Potential Brute-Force Attack Sources:", brute_force_ips if brute_force_ips else "None detected")
print("Potential Failed Login Attack Sources:", failed_login_ips if failed_login_ips else "None detected")
print("=====================================")


========= DETECTION REPORT =========
Potential Brute-Force Attack Sources: ['10.11.12.101', '80.85.159.236', '194.87.147.244', '124.217.255.96', '94.140.114.6']
Potential Failed Login Attack Sources: None detected


In [4]:

from scapy.all import rdpcap, DNSQR
from collections import Counter

# Path to the pcap file (update this to your actual file path)
pcap_file_path = "/content/Sample.pcap"

# Read packets from the pcap file
packets = rdpcap(pcap_file_path)

# Extract DNS query names
dns_queries = []

for packet in packets:
    if packet.haslayer(DNSQR):  # Check if packet contains a DNS query
        dns_queries.append(packet[DNSQR].qname.decode())  # Extract domain name

# Count the most frequently queried domains
dns_counts = Counter(dns_queries)

# Display the top queried domains
print("Most Frequently Queried Domains:")
for domain, count in dns_counts.most_common(10):  # Show top 10
    print(f"{domain}: {count} times")

Most Frequently Queried Domains:
wpad.localdomain.: 68 times
wpad.: 44 times
isatap.localdomain.: 42 times
teredo.ipv6.microsoft.com.: 12 times
Smithers-PC.: 11 times
google.com.: 6 times
www.google.com.: 4 times
kh2714ldb.com.: 4 times
gmail.com.: 4 times
dns.msftncsi.com.: 2 times
